# 🚀 Supervised Fine-Tuning on Network Flow Dataset
This notebook tokenizes network flow samples and fine-tunes Mistral-7B with a classification label (e.g., Attack/Normal).

In [1]:
from datasets import load_dataset

# Load dataset from Hugging Face
dataset = load_dataset("Suvo696/InSDNN", split="train")

# Only use 1% of data for initial testing
sample_size = int(0.01 * len(dataset))
dataset = dataset.select(range(sample_size))

# Show one example to verify
print(dataset[0])

/Users/apple/Documents/InSDN-Experiments/workenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'Flow ID': '192.168.3.130-200.175.2.130-80-43226-6', 'Src IP': '200.175.2.130', 'Src Port': 43226, 'Dst IP': '192.168.3.130', 'Dst Port': 80, 'Protocol': 6, 'Timestamp': '9/1/2020 16:30', 'Flow Duration': 15163, 'Tot Fwd Pkts': 4, 'Tot Bwd Pkts': 4, 'TotLen Fwd Pkts': 30.0, 'TotLen Bwd Pkts': 30.0, 'Fwd Pkt Len Max': 30, 'Fwd Pkt Len Min': 0, 'Fwd Pkt Len Mean': 7.5, 'Fwd Pkt Len Std': 15.0, 'Bwd Pkt Len Max': 30, 'Bwd Pkt Len Min': 0, 'Bwd Pkt Len Mean': 7.5, 'Bwd Pkt Len Std': 15.0, 'Flow Byts/s': 3957.000594, 'Flow Pkts/s': 527.6000791, 'Flow IAT Mean': 2166.142857, 'Flow IAT Std': 2270.582776, 'Flow IAT Max': 5461.0, 'Flow IAT Min': 12.0, 'Fwd IAT Tot': 10762.0, 'Fwd IAT Mean': 3587.333333, 'Fwd IAT Std': 1975.794102, 'Fwd IAT Max': 5634.0, 'Fwd IAT Min': 1691.0, 'Bwd IAT Tot': 8011.0, 'Bwd IAT Mean': 2670.333333, 'Bwd IAT Std': 4496.549603, 'Bwd IAT Max': 7862.0, 'Bwd IAT Min': 12.0, 'Fwd PSH Flags': 0, 'Bwd PSH Flags': 0, 'Fwd URG Flags': 0, 'Bwd URG Flags': 0, 'Fwd Header Len':

In [2]:
from transformers import AutoTokenizer

# Load tokenizer
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token 

# Optional: Keep only 1% of data for quick training
small_dataset = dataset

# Tokenize function (input: full text, label: only Attack/Normal string)
def tokenize_row(example):
    input_enc = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=1152
    )
    target_enc = tokenizer(
        example["Label"],
        truncation=True,
        padding="max_length",
        max_length=10
    )

    # Set up labels, ignoring input token loss by using -100
    input_enc["labels"] = [-100] * len(input_enc["input_ids"])
    input_enc["labels"][:len(target_enc["input_ids"])] = target_enc["input_ids"]

    return input_enc

# Apply to dataset
tokenized_dataset = small_dataset.map(tokenize_row)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/apple/Documents/InSDN-Experiments/workenv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/apple/Documents/InSDN-Experiments/workenv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/apple/Documents/InSDN-Experiments/workenv/lib/python3.11/site-packages/ipykernel/

KeyboardInterrupt: 